<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/RH_popularity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Import packages

In [28]:
import os
import pandas as pd
import datetime as dt
import wrds

# Combine files

## Iterating over files

In [2]:
files = [x for x in os.listdir('/content/') if '.csv' in x]
files.sort()
ticker = [x.rsplit('.csv')[0] for x in files]

In [3]:
#def chunks(lst, n):
#    """Yield successive n-sized chunks from lst."""
#    for i in range(0, len(lst), n):
#        yield lst[i:i + n]

This function can be used to devide the file list into chunks. It might come in handy to run script on the **8,597** files.

## Appending data

In [4]:
time = []
users = []
tic = []
for i in range(len(files)):
    filepath = '/content'
    pos = os.path.join(filepath, files[i])
    read = pd.read_csv(pos, sep=',', header=0)
    time.extend(list(read.iloc[:,0]))
    users.extend(list(read.iloc[:,1]))
    tic.extend([ticker[i]] * len(read))

## Reading data into dataframe

In [5]:
data = pd.DataFrame({'tic':tic, 'timestamp':time, 'users_holding':users})

## Change *timestamp* to datetime type

In [6]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Summarize data to daily intervals

In [7]:
data['date'] = data['timestamp'].dt.date

In [8]:
g = data[['tic','date','users_holding']].groupby(['tic','date'])

In [9]:
data[data['tic']=='WOOD']

,tic,timestamp,users_holding,date


In [10]:
daily = g.users_holding.aggregate(['first','last','min','max'])

In [19]:
daily.loc['ADBE',:]

,first,last,min,max
date,,,,
2018-05-02,8007.0,8007.0,8007.0,8007.0
2018-05-03,8030.0,7980.0,7980.0,8035.0
2018-05-04,7980.0,7916.0,7916.0,7980.0
2018-05-05,7916.0,7916.0,7916.0,7916.0
2018-05-06,7916.0,7916.0,7916.0,7916.0
...,...,...,...,...
2020-08-09,23737.0,23737.0,23737.0,23737.0
2020-08-10,23737.0,23994.0,23737.0,23994.0
2020-08-11,23994.0,24007.0,23988.0,24024.0


In [22]:
by_tic = daily.reset_index().groupby('tic')

In [26]:
by_tic['first'].agg('count')

tic
A        818
AA       600
AAAU     714
AACAY    701
AACG     818
        ... 
BPOP     818
BPRN     818
BPT      818
_OUT     818
_PRN     817
Name: first, Length: 960, dtype: int64

# Variable construction

 - Daily change: $\mid$ `last`-`first` $\mid$
 - Daily activity: $\mid$ `max` - `min` $\mid$

Signed change and activity might be more suitable if we extract signed (e.g. positivity vs. negativity) data from WSB.

# Grander causality

## Run panel Granger using *R*
[Dumitrescu-Hurlin (2012) in plm](https://rdrr.io/cran/plm/man/pgrangertest.html): There is no predetermined way to set the lags.

# Lead-lag relation
Timing of information and actual trading

$WSB_t=\beta_0+ \sum_{\substack{{}\\t-3}}^{t+3} \beta_n\times \Delta ROBINHOOD_n
$

$\Delta ROBINHOOD_t=\beta_0+ \sum_{\substack{{}\\t-3}}^{t+3} \beta_n\times WSB_n$

- openning
- closing
- min
- max

- Type of firms
- Institutional holdings

# CRSP
- Bid-ask
- *High-low*

other than returns; during trading hours